In [1]:
import gym
import tensorflow as tf
import numpy as np
import random
import matplotlib.pyplot as plt


In [2]:
env = gym.make('CartPole-v1')

In [3]:
#param

LEARNING_RATE = 0.001
GAMMA = 0.99

#EXPLORATION_MAX = 1.0 # epsilon greedy
#EXPLORATION_MIN = 0.01
#EXPLORATION_DECAY = 0.995

MEMORY_SIZE = 100 #1000000 #50000
#BATCH_SIZE = 100 #20

In [71]:
class Agent(tf.keras.Model):

  def __init__(self):
    super(Agent, self).__init__()
    # Define your layers here.
    self.dense1 = tf.keras.layers.Dense(32, activation='relu')
    #self.dropout1 = tf.keras.layers.Dropout(0.5)
    #self.dense2 = tf.keras.layers.Dense(24, activation='relu')
    #self.dropout2 = tf.keras.layers.Dropout(0.5)
    self.dense2 = tf.keras.layers.Dense(2, activation='softmax')

  def call(self, inputs):
    tmp = tf.convert_to_tensor(inputs)
    tmp = self.dense1(tmp)
    tmp = self.dense2(tmp)
    
    logits = tmp
    logs_pi = tf.math.log(logits)
    return logits, logs_pi

  def explore(self, inputs):
    logits, logs_pi = self(inputs)
    return tf.squeeze(tf.random.categorical(logits, 1)), tf.squeeze(logs_pi)

  def exploit(self, inputs):
    logits, _ = self(inputs)
    return tf.math.argmax(logits)




In [72]:
agent = Agent()
#agent.compile(optimizer=tf.keras.optimizers.Adam())
agent.build(tf.TensorShape([None,4]))
agent.compile(optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
    loss="mse",
    metrics=['accuracy'])

agent.summary()

Model: "agent_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              multiple                  160       
_________________________________________________________________
dense_9 (Dense)              multiple                  66        
Total params: 226
Trainable params: 226
Non-trainable params: 0
_________________________________________________________________


In [6]:
#def showHistory(history) :
#    #print(history.history.keys())
#    plt.figure(figsize=(15.0,10.0))
#    fig, axes = plt.subplots(nrows=1, ncols=2) 
#    fig.set_size_inches(15.0, 7.0)         
#    axes[0].plot(history.history['loss'], label="loss")
#    axes[0].legend()
#    axes[1].plot(history.history['accuracy'], label="accuracy")
#    axes[1].legend()
#    plt.show()


In [7]:
##env.close()
##agent.save_weights('CartPoleV1.tf')
##agent.load_weights('CartPoleV1.tf')
#
#def drawAgent(agent, p=0.0, dp=0.0) :
#    rx = np.arange(-0.24,0.24,0.02, np.float32)
#    ry = np.arange(-2,2,0.2, np.float32)
#    def agentDir(x,y) :
#        [a ,b] = agent([[p, dp, x, y]])[0]
#        #return '<' if b<a else '>'
#        #return (b-a).numpy()
#        return np.sign(b-a)
#        
#    pix = [
#        #[ [[type(0.0), type(0.0), type(x), type(y)]] for x in rx]
#        [ agentDir(x, y) for x in rx]
#        for y in ry
#        ]
#    #print(pix)
#    #return
#    fig, ax = plt.subplots()
#    fig.set_size_inches(15.0, 7.0)         
#    im = ax.imshow(pix)
#    ax.set_xticks(np.arange(len(rx)))
#    ax.set_yticks(np.arange(len(ry)))
#    ax.set_xticklabels([round(l,2) for l in rx])
#    ax.set_yticklabels([round(l,2) for l in ry])
#    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")    
#    for y in range(len(ry)):
#        for x in range(len(rx)):
#            #text = ax.text(x, y, pix[y][x], ha="center", va="center", color="w")
#            text = ax.text(x, y, '<' if pix[y][x]<0 else '>' if pix[y][x]>0 else '0', ha="center", va="center", color="w")
#    plt.show()
#
##print(pix)


In [8]:
memory = [] #{ "states":[], "masks":[], "actions":[], "values":[]}
#loss = []
#accuracy = []
#eg = EXPLORATION_MAX
#scores = []

In [ ]:
for epoch in range(1,5000) :
    print("Epoch : ", epoch)
    state = env.reset()
    
    step = 0
    epoch_memory = [] #np.array()
    #for step in range(1000):
    while True :
        step += 1
        #if eg<0.03 :
        env.render()
        #    #env.render()
        #    pass
        #input()
        #if (random.random() > eg) :
        #    Qs = agent([state])[0];
        #    action = np.argmax(Qs);
            #Q = Qs[action]
            #print("State: ", state, " Qs: ", Qs.numpy(), " Action: ", action, " Q: ", Q.numpy())
        #else :
            #Qs = agent([state])[0];
        #    action = random.randrange(2)
            #Q = Qs[action]
            #print("State: ", state, " Action: ", action, " Q: ", Q.numpy())
        #print(state)
        action, logs_pi = agent.explore([state])
        #print(action)
        #action = action[0].numpy(); 
        #logs_pi = logs_pi[0] 
        #print(action)
        #print(logs_pi)

        state_next, reward, done, info = env.step(action.numpy())
        if done : reward = -1.0
        
        epoch_memory.append({ "s": state, "a": action, "r": reward, "l": logs_pi, "d": done, "sn": state_next })
        
        #mem = { "s": state, "a": action, "r": reward, "d": done, "sn": state_next, }        
        #memory.append(mem)
        
        state = state_next

        if done :
            reward = 0
            for mem in numpy.flip(epoch_memory) :
                reward = reward * GAMMA + mem['r']
                mem['r'] = reward
            
            memory.extend(epoch_memory)
            #print('-' if abs(state[0])>2.4 else 'a' if abs(state[2])>0.2094 else '?')
            print(' ', reward)
            break
        else :
            print('.', end='')
        
        state = state_next
        
    if True or len(memory) >= MEMORY_SIZE :
        
        tf_adv = [ mem['r'] for mem in memory ]
        
        #logp_a_op = tf.reduce_sum(tf.one_hot(tf_a, depth=action_space) * logp_all, axis=1)
        logp_a_op = [ mem['l'][mem['a']] for mem in memory ]
        
        #self.pi_loss = PolicyGradient.net_objectives(
        #    tf_adv=self.tf_adv,
        #    logp_a_op=self.logp_a_op
        #)
        pi_loss = -tf.reduce_mean(logp_a_op * tf_adv)
        
        train_pi = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE).minimize(pi_loss)
        
        approx_ent = tf.reduce_mean(-logp_a_op)
        
        print(train_pi.numpy(), approx_ent.numpy())
        
        
    break
    #drawAgent(agent)
    #scores.append(step)
    #print("step : ", step, " (", np.min(scores), " / ", np.average(scores[max(0,len(scores)-100):]), " / ", np.max(scores), ")")
    #print(memory)
    #eg = max(EXPLORATION_MIN, eg * EXPLORATION_DECAY)
    #print("\r\neG: ", eg, "; ", c0,"-",c1,"/",c0+c1)
    #print("\r\neG: ", eg)
    
    #plt.figure(figsize=(15.0,7.0))
    #fig, axes = plt.subplots(nrows=1, ncols=2) 
    #fig.set_size_inches(15.0, 7.0)         
    #axes[0].plot(loss, label="loss")
    #axes[0].legend()
    #axes[1].plot(accuracy, label="accuracy")
    #axes[1].legend()
    #plt.show()
    #loss = []
    #accuracy = []

    #showHistory(history)
    
    #print(agent([s0])[0].numpy(), " <= ", o0)
    #break
    
    
    

In [ ]:
#eg = 0.05
env.close()
#plt.plot(scores)
#plt.show()

In [42]:
toto = tf.random.categorical([[0.2 ,0.2, 0.6],[0.2 ,0.2, 0.6]], 1)
print (toto)

tf.Tensor(
[[0]
 [2]], shape=(2, 1), dtype=int64)
